In [1]:
import investpy

In [35]:
# Data manipulation modules
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import yfinance as yf

# Data visualization modules
import matplotlib.pyplot as plt

# Machine Learning modules
from keras.models import Sequential
from keras.layers import Activation, Dense

# Cross-validation modules 
from sklearn.model_selection import TimeSeriesSplit

# Grid search module
from sklearn.model_selection import GridSearchCV

# Evaluation metrics modules
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.metrics import Precision

In [36]:
def get_train_test_data(cripto_series, test_percentage):
    '''This functions creates the training and test data for the stock time series'''
    time_series_length = len(cripto_series)
    test_series = cripto_series.iloc[int(time_series_length*(1-test_percentage)):time_series_length]
    train_series = cripto_series.iloc[:-int(time_series_length*test_percentage)]
    
    return(test_series, train_series)

def get_rate_of_change(cripto_series, variable, n_days):
    '''This function computes several values for the rate of change and add them into a dataset'''
    for day in n_days: 
        variable_name = f"{day}_days_of_change"
        cripto_series[variable_name] = (cripto_series[variable] - cripto_series[variable].shift(day)) / cripto_series[variable].shift(day)
        
def get_moving_average(cripto_series, variable, n_days):
    '''This function computes several values for the moving average and add them into a dataset'''
    for day in n_days:
        variable_name = f"{day}_day_moving_average"
        cripto_series[variable_name] = cripto_series[variable].rolling(day).mean()
        
def get_ratio(cripto_series, variable_1, variable_2):
    '''This function computes the ratio between two variables and add it into a dataset'''
    variable_name = f"{variable_1}_{variable_2}_ratio"
    cripto_series[variable_name] = cripto_series[variable_1] / cripto_series[variable_2]
    
def get_difference(cripto_series, variable_1, variable_2):
    '''This function computes the difference between the values of two variables and add it into a dataset'''
    variable_name = f"{variable_1}_{variable_2}_difference"
    cripto_series[variable_name] = cripto_series[variable_1] - cripto_series[variable_2] 
    
def train_and_predict(model, train_predictors, train_target, test_predictors):
    '''This function fits a model and then make predictions on it for a given amount of data'''
    model.fit(train_predictors, train_target)
    preds = model.predict(test_predictors)
    preds = pd.Series(preds, index=test_series_2.index)
    return preds

In [37]:
stock_series = investpy.stocks.get_stock_historical_data('PETR4', 'brazil', from_date = '01/01/2002', to_date='01/08/2022')

In [38]:
stock_series.head()

,Open,High,Low,Close,Volume,Currency
Date,,,,,,
2002-01-02,4.73,4.79,4.72,4.73,23979552,BRL
2002-01-03,4.75,4.80,4.74,4.76,39124800,BRL
2002-01-04,4.76,4.76,4.67,4.73,21548832,BRL
2002-01-07,4.71,4.87,4.71,4.83,40547104,BRL
2002-01-08,4.83,4.83,4.77,4.80,34469888,BRL


In [39]:
test_series, train_series = get_train_test_data(stock_series, test_percentage=0.1)

## Applying the new variables

In [40]:
train_series['target']  = np.array(train_series['Close'].shift(-1) > train_series['Close']).astype(int) 

In [41]:
get_rate_of_change(train_series, 'Close', [3, 5, 7, 15])

In [42]:
get_difference(train_series, 'Close', 'Open')
get_difference(train_series, 'High', 'Low')

In [43]:
train_series['low_dif'] = (train_series['Low'] - train_series['Low'].shift(1))
train_series['high_dif'] = train_series['High'] - train_series['High'].shift(1)

In [44]:
get_ratio(train_series, 'Open', 'Close')
get_ratio(train_series, 'High', 'Close')
get_ratio(train_series, 'Low', 'Close')

In [45]:
train_series.drop('Currency', axis=1, inplace=True)

In [46]:
train_series.dropna(axis=0, inplace=True)

In [47]:
test_series['target']  = np.array(test_series['Close'].shift(-1) > test_series['Close']).astype(int) 

In [48]:
get_rate_of_change(test_series, 'Close', [3, 5, 7, 15])

In [49]:
get_difference(test_series, 'Close', 'Open')
get_difference(test_series, 'High', 'Low')

In [50]:
test_series['low_dif'] = (test_series['Low'] - test_series['Low'].shift(1))
test_series['high_dif'] = test_series['High'] - test_series['High'].shift(1)

In [51]:
get_ratio(test_series, 'Open', 'Close')
get_ratio(test_series, 'High', 'Close')
get_ratio(test_series, 'Low', 'Close')

In [52]:
test_series.drop('Currency', axis=1, inplace=True)

In [53]:
test_series.dropna(axis=0, inplace=True)

In [54]:
print(train_series.columns)
print(test_series.columns)

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'target', '3_days_of_change',
       '5_days_of_change', '7_days_of_change', '15_days_of_change',
       'Close_Open_difference', 'High_Low_difference', 'low_dif', 'high_dif',
       'Open_Close_ratio', 'High_Close_ratio', 'Low_Close_ratio'],
      dtype='object')
Index(['Open', 'High', 'Low', 'Close', 'Volume', 'target', '3_days_of_change',
       '5_days_of_change', '7_days_of_change', '15_days_of_change',
       'Close_Open_difference', 'High_Low_difference', 'low_dif', 'high_dif',
       'Open_Close_ratio', 'High_Close_ratio', 'Low_Close_ratio'],
      dtype='object')


In [55]:
predictors = ['Open', 'High', 'Low', 'Close', 'Volume',
       '3_days_of_change', '5_days_of_change', '7_days_of_change',
       '15_days_of_change', 'Close_Open_difference', 'High_Low_difference',
       'low_dif', 'high_dif', 'Open_Close_ratio', 'High_Close_ratio',
       'Low_Close_ratio']

In [56]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(train_series[predictors].shape[1:])))
model.add(Dense(units=10, activation='relu', input_shape=(train_series[predictors].shape[1:])))
model.add(Dense(units=6, activation='relu', input_shape=(train_series[predictors].shape[1:])))
model.add(Dense(1, activation = 'sigmoid'))

In [57]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', Precision()])

In [58]:
model.fit(train_series[predictors], train_series['target'], batch_size=10,epochs=100)

Epoch 1/100
458/458 [==============================] - 2s 2ms/step - loss: 22434.6641 - accuracy: 0.5047 - precision_1: 0.5162
Epoch 2/100
458/458 [==============================] - 1s 2ms/step - loss: 5307.6768 - accuracy: 0.5003 - precision_1: 0.5115
Epoch 3/100
458/458 [==============================] - 1s 2ms/step - loss: 4402.8418 - accuracy: 0.5060 - precision_1: 0.5167
Epoch 4/100
458/458 [==============================] - 2s 3ms/step - loss: 3378.8989 - accuracy: 0.4975 - precision_1: 0.5086
Epoch 5/100
458/458 [==============================] - 1s 3ms/step - loss: 2984.7344 - accuracy: 0.5014 - precision_1: 0.5126
Epoch 6/100
458/458 [==============================] - 1s 2ms/step - loss: 2710.0400 - accuracy: 0.4992 - precision_1: 0.5103
Epoch 7/100
458/458 [==============================] - 1s 3ms/step - loss: 3016.9048 - accuracy: 0.4977 - precision_1: 0.5090
Epoch 8/100
458/458 [==============================] - 1s 2ms/step - loss: 1808.7072 - accuracy: 0.4903 - precision_1

458/458 [==============================] - 1s 2ms/step - loss: 0.6929 - accuracy: 0.5115 - precision_1: 0.5115
Epoch 67/100
458/458 [==============================] - 1s 3ms/step - loss: 0.6930 - accuracy: 0.5115 - precision_1: 0.5115
Epoch 68/100
458/458 [==============================] - 1s 3ms/step - loss: 0.6929 - accuracy: 0.5115 - precision_1: 0.5115
Epoch 69/100
458/458 [==============================] - 1s 2ms/step - loss: 0.6929 - accuracy: 0.5115 - precision_1: 0.5115
Epoch 70/100
458/458 [==============================] - 1s 2ms/step - loss: 0.6929 - accuracy: 0.5115 - precision_1: 0.5115
Epoch 71/100
458/458 [==============================] - 1s 2ms/step - loss: 0.6929 - accuracy: 0.5115 - precision_1: 0.5115
Epoch 72/100
458/458 [==============================] - 1s 3ms/step - loss: 0.6929 - accuracy: 0.5115 - precision_1: 0.5115
Epoch 73/100
458/458 [==============================] - 1s 3ms/step - loss: 0.6929 - accuracy: 0.5115 - precision_1: 0.5115
Epoch 74/100
458/458 

In [59]:
model.evaluate(test_series[predictors], test_series['target'])

16/16 [==============================] - 1s 2ms/step - loss: 0.6927 - accuracy: 0.5152 - precision_1: 0.5152


[0.692701518535614, 0.5151515007019043, 0.5151515007019043]

In [60]:
preds = model.predict(test_series[predictors])
preds = preds.reshape(-1)
preds = pd.Series(preds, index=test_series.index)

16/16 [==============================] - 0s 1ms/step


In [61]:
preds[preds >= 0.5] = 1
preds[preds < 0.5] = 0

In [62]:
precision_score(test_series['target'], preds)

0.5151515151515151

In [63]:
recall_score(test_series['target'], preds)

1.0

In [64]:
accuracy_score(test_series['target'], preds)

0.5151515151515151

In [65]:
confusion_matrix(test_series['target'], preds)

array([[  0, 240],
       [  0, 255]], dtype=int64)